In [1]:
import os
import pickle
import pandas as pd
import numpy as np

pnum_list = [1, 4, 5, 8, 9, 10, 11, 13, 14, 15, 16, 19, 22, 23, 24, 25, 26, 27, 28, 31, 32]

for pnum in pnum_list:
    ann_path = os.getcwd() + '/dataset/K-EmoCon/raw/emotion_annotations/self_annotations/P' + str(pnum) + '.self.csv'
    ann_file = pd.read_csv(ann_path)
    ann_file = ann_file.iloc[:, :3]

    pnum_to_raw_df ={}
    e4_dir = os.getcwd() + f'/dataset/K-EmoCon/raw/e4_data/{pnum}/'
    e4_paths = [os.path.join(e4_dir, f) for f in os.listdir(e4_dir) if f != '.ipynb_checkpoints']
    for e4_path in e4_paths:
        filetype = e4_path.split('/')[-1].split('_')[-1].split('.')[0].lower()
        filekey = f'{pnum}/{filetype}'
        data = pd.read_csv(e4_path)

        # take care of multi-entry issue
        if pnum == 31 and filetype == 'ibi':
            data = data.loc[data.device_serial == 'A01525']
        elif pnum == 31:
            data = data.loc[data.device_serial == 'A013E1']
        elif pnum == 32:
            data = data.loc[data.device_serial == 'A01A3A']

        pnum_to_raw_df[filekey] = data
    
    h7_dir = os.getcwd() + f'/dataset/K-EmoCon/raw/neurosky_polar_data/{pnum}/Polar_HR.csv'
    filekey = f'{pnum}/ecg'
    pnum_to_raw_df[filekey] = pd.read_csv(h7_dir)

    brain_dir = os.getcwd() + f'/dataset/K-EmoCon/raw/neurosky_polar_data/{pnum}/BrainWave.csv'
    filekey = f'{pnum}/eeg'
    pnum_to_raw_df[filekey] = pd.read_csv(brain_dir)

    pnum_info_table = pd.read_csv(os.getcwd() + '/dataset/K-EmoCon/metadata/subjects.csv', index_col='pid')
    pnum_info_table = pnum_info_table.loc[pnum_info_table.index==pnum]
    init_time, start_time, end_time = tuple(pnum_info_table[['initTime', 'startTime', 'endTime']].to_numpy()[0])

    e4_hr_df = pnum_to_raw_df[f'{pnum}/hr'][['timestamp', 'value']]
    e4_eda_df = pnum_to_raw_df[f'{pnum}/eda'][['timestamp','value']]
    e4_acc_df = pnum_to_raw_df[f'{pnum}/acc'][['timestamp', 'x','y','z']]
    e4_temp_df = pnum_to_raw_df[f'{pnum}/temp'][['timestamp', 'value']]
    e4_bvp_df = pnum_to_raw_df[f'{pnum}/bvp'][['timestamp','value']]
    ecg_df = pnum_to_raw_df[f'{pnum}/ecg'][['timestamp','value']]
    ecg_df.index = pd.DatetimeIndex(ecg_df.timestamp * 1e6)
    ecg_df = ecg_df.resample('1S').mean()
    ecg_df.interpolate(method='time', inplace=True)
    ecg_df.index = ecg_df.index.astype(np.int64) // 1e6
    eeg_df = pnum_to_raw_df[f'{pnum}/eeg'][['timestamp','lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','middleGamma','delta','theta']]

    dict_data = {'signal':
                 {
                    # 'e4_hr': e4_hr_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6), 'value'].to_numpy(),
                    'e4_eda': e4_eda_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6), ['value']].to_numpy(),
                    'e4_acc': e4_acc_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6), ['x','y','z']].to_numpy(),
                    'e4_temp': e4_temp_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6), ['value']].to_numpy(),
                    'e4_bvp': e4_bvp_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6), ['value']].to_numpy(),
                    # 'ecg_hr': ecg_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6), 'value'].to_numpy(),
                    # 'eeg': eeg_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6), ['lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','middleGamma','delta','theta']].to_numpy(),
                  }, 'label':
                  {
                    'arousal':ann_file['arousal'].to_numpy(),
                    'valence':ann_file['valence'].to_numpy()
                  }}    
    
    folder_name = f"./dataset/FINAL/S{pnum}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    pkl_file_path = os.path.join(folder_name, f"S{pnum}.pkl")
    with open(pkl_file_path, 'wb') as pkl_file:
        pickle.dump(dict_data, pkl_file)

In [14]:
import os
import pickle
import pandas as pd
import numpy as np

pnum_list = [1, 4, 5, 8, 9, 10, 11, 13, 14, 15, 16, 19, 22, 23, 24, 25, 26, 27, 28, 31, 32]

for pnum in pnum_list:
    ann_path = os.getcwd() + '/dataset/K-EmoCon/raw/emotion_annotations/self_annotations/P' + str(pnum) + '.self.csv'
    ann_file = pd.read_csv(ann_path)
    ann_file = ann_file.iloc[:, :3]

    pnum_to_raw_df ={}
    e4_dir = os.getcwd() + f'/dataset/K-EmoCon/raw/e4_data/{pnum}/'
    e4_paths = [os.path.join(e4_dir, f) for f in os.listdir(e4_dir) if f != '.ipynb_checkpoints']
    for e4_path in e4_paths:
        filetype = e4_path.split('/')[-1].split('_')[-1].split('.')[0].lower()
        filekey = f'{pnum}/{filetype}'
        data = pd.read_csv(e4_path)

        # take care of multi-entry issue
        if pnum == 31 and filetype == 'ibi':
            data = data.loc[data.device_serial == 'A01525']
        elif pnum == 31:
            data = data.loc[data.device_serial == 'A013E1']
        elif pnum == 32:
            data = data.loc[data.device_serial == 'A01A3A']

        pnum_to_raw_df[filekey] = data
    
    h7_dir = os.getcwd() + f'/dataset/K-EmoCon/raw/neurosky_polar_data/{pnum}/Polar_HR.csv'
    filekey = f'{pnum}/ecg'
    pnum_to_raw_df[filekey] = pd.read_csv(h7_dir)

    brain_dir = os.getcwd() + f'/dataset/K-EmoCon/raw/neurosky_polar_data/{pnum}/BrainWave.csv'
    filekey = f'{pnum}/eeg'
    pnum_to_raw_df[filekey] = pd.read_csv(brain_dir)

    pnum_info_table = pd.read_csv(os.getcwd() + '/dataset/K-EmoCon/metadata/subjects.csv', index_col='pid')
    pnum_info_table = pnum_info_table.loc[pnum_info_table.index==pnum]
    init_time, start_time, end_time = tuple(pnum_info_table[['initTime', 'startTime', 'endTime']].to_numpy()[0])

    e4_hr_df = pnum_to_raw_df[f'{pnum}/hr']
    e4_eda_df = pnum_to_raw_df[f'{pnum}/eda']
    e4_acc_df = pnum_to_raw_df[f'{pnum}/acc']
    e4_temp_df = pnum_to_raw_df[f'{pnum}/temp']
    e4_bvp_df = pnum_to_raw_df[f'{pnum}/bvp']
    ecg_df = pnum_to_raw_df[f'{pnum}/ecg']
    ecg_df.index = pd.DatetimeIndex(ecg_df.timestamp * 1e6)
    ecg_df = ecg_df.resample('1S').mean()
    ecg_df.interpolate(method='time', inplace=True)
    ecg_df.index = ecg_df.index.astype(np.int64) // 1e6
    eeg_df = pnum_to_raw_df[f'{pnum}/eeg']

    e4_hr_df = e4_hr_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6)]
    e4_eda_df = e4_eda_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6)]
    e4_acc_df = e4_acc_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6)]
    e4_temp_df = e4_temp_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6)]
    e4_bvp_df = e4_bvp_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6)]
    ecg_df = ecg_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6)]
    eeg_df = eeg_df.loc[lambda x: (x.timestamp >= start_time) & (x.timestamp < end_time + 1e6)]

    dict_data = {'signal':
                 {
                    # 'e4_hr': e4_hr_df[['value']].to_numpy(),
                    'e4_eda': e4_eda_df[['value']].to_numpy(),
                    'e4_acc': e4_acc_df[['x','y','z']].to_numpy(),
                    'e4_temp': e4_temp_df[['value']].to_numpy(),
                    'e4_bvp': e4_bvp_df[['value']].to_numpy(),
                    # 'ecg_hr': ecg_df[['value']].to_numpy(),
                    # 'eeg': eeg_df[['lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','middleGamma','delta','theta']].to_numpy(),
                  }, 'label':
                  {
                    'arousal':ann_file[['arousal']].to_numpy(),
                    'valence':ann_file[['valence']].to_numpy()
                  }}    
    
    folder_name = f"./dataset/FINAL/S{pnum}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    pkl_file_path = os.path.join(folder_name, f"S{pnum}.pkl")
    with open(pkl_file_path, 'wb') as pkl_file:
        pickle.dump(dict_data, pkl_file)

In [12]:
pnum_to_raw_df

{'1/hr':          timestamp  pid  value device_serial  device_number     entry_time
 0     1.548120e+12    1  71.00        A01A3A              5  1548120108000
 1     1.548120e+12    1  71.00        A01A3A              5  1548120108000
 2     1.548120e+12    1  71.00        A01A3A              5  1548120108000
 3     1.548120e+12    1  71.75        A01A3A              5  1548120108000
 4     1.548120e+12    1  72.20        A01A3A              5  1548120108000
 ...            ...  ...    ...           ...            ...            ...
 1560  1.548122e+12    1  77.75        A01A3A              5  1548120108000
 1561  1.548122e+12    1  77.70        A01A3A              5  1548120108000
 1562  1.548122e+12    1  77.67        A01A3A              5  1548120108000
 1563  1.548122e+12    1  77.77        A01A3A              5  1548120108000
 1564  1.548122e+12    1  77.87        A01A3A              5  1548120108000
 
 [1565 rows x 6 columns],
 '1/eda':          timestamp  pid     value device_s

In [10]:
dict_data['signal']['eeg'].shape

(659, 8)

In [13]:
pd.read_csv(brain_dir)

,timestamp,delta,lowAlpha,highAlpha,lowBeta,highBeta,lowGamma,middleGamma,theta,isValid,pid
0,1548120111526,1075,166,1158,1627,179,246,8657,3664,1,1
1,1548120112540,11468,18089,6835,11808,716,614,321359,32559,1,1
2,1548120113597,2,0,0,1,1,0,2,2,1,1
3,1548120114529,0,0,1,1,0,0,1,0,1,1
4,1548120115513,5,28,20,8,2,6,143,7,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1562,1548121667260,5117,2797,11521,9366,1155,1226,42230,33747,1,1
1563,1548121668261,24726,164470,17925,81387,3399,4958,431170,134633,1,1
1564,1548121669264,83612,16047,45715,24382,2255,2090,133612,190579,1,1
1565,1548121670267,254464,148092,183552,608043,30989,54663,1702569,2106455,1,1
